Off-the-shelf method using Surprise library from scikit for collaborative filtering.

The Surprise library offers matrix factorization-based prediction algorithms such as SVD, SVD++, NMF.

In [1]:
import numpy as np
from surprise import Dataset, evaluate, Reader, accuracy
from surprise.prediction_algorithms import predictions, matrix_factorization
from surprise.model_selection import cross_validate, GridSearchCV

In [2]:
# load train and test data
r = Reader(sep='\t')
train_data = Dataset.load_from_file('data/train_new.txt', reader=r)
train_Y = train_data.build_full_trainset()

test_data = Dataset.load_from_file('data/test.txt', reader=r)
test_Y = test_data.build_full_trainset()
test_data = test_Y.build_testset()

In [ ]:
# SVD
svd = matrix_factorization.SVD()

# SVD++ algorithm aka extension of SVD taking into account implicit ratings
svdpp = matrix_factorization.SVDpp()

# Non-negative Matrix factorization
nmf = matrix_factorization.NMF()

# Fit and train, compute root mean square error
for model in [svd, svdpp, nmf]:
    model.fit(train_Y)
    print(model)
    accuracy.rmse(model.test(test_data)) # Then compute RMSE
    print()
    
# Results: (SVD++ algo had best RMSE)
# <surprise.prediction_algorithms.matrix_factorization.SVD object at 0x0000015C1957C240>
# RMSE: 0.9316

# <surprise.prediction_algorithms.matrix_factorization.SVDpp object at 0x0000015C1957C9E8>
# RMSE: 0.9098

# <surprise.prediction_algorithms.matrix_factorization.NMF object at 0x0000015C1957CBE0>
# RMSE: 0.9566

In [3]:
k = 20
eta = 0.1
reg = 0.1

model = matrix_factorization.SVDpp(n_factors=k, lr_all=eta, reg_all=reg, verbose=True)
model.fit(train_Y)
accuracy.rmse(model.test(test_data)) # epochs: 19, RMSE: 0.9639059164365897 on predictions

 processing epoch 0
 processing epoch 1
 processing epoch 2
 processing epoch 3
 processing epoch 4
 processing epoch 5
 processing epoch 6
 processing epoch 7
 processing epoch 8
 processing epoch 9
 processing epoch 10
 processing epoch 11
 processing epoch 12
 processing epoch 13
 processing epoch 14
 processing epoch 15
 processing epoch 16
 processing epoch 17
 processing epoch 18
 processing epoch 19
RMSE: 0.9639


0.9639059164365897

In [9]:
'''
This function performs matrix factorization.
Input:
    Y_train: training labels
    test_set: test set
Output: 
    newU: The 2D version of U
    newV: The 2D version of V
'''
def matrix_factor(model, train_Y):
    """ Uses matrix factorization-based algorithm SVD++
    and returns U (k x m matrix) and V (k x n matrix)
    where k in the number of factors of the model. """    
    # get user factors
    U = model.pu    
    print("U shape: " + str(U.shape)) # (m, k) = (943, 20)

    # transpose item factors
    V = np.transpose(model.qi)    
    print("V shape: " + str(V.shape)) # (k, n) = (20, 1682)
    
    return U, V
    
def SVD(U, V):
    """ Applies SVD to V and uses the ﬁrst two columns of A
    to project U,V into 2-D space. """    
    # get A matrix
    A, _, _ = np.linalg.svd(V) 
    # take first two columns of A
    A = A[:, [0, 1]] # (k, 2) = (5, 2)
    print("A shape: " + str(A.shape))

    # project U into 2-D space
    U_tilda = np.transpose(np.dot(np.transpose(A), np.transpose(U)))
    print("U shape: " + str(U_tilda.shape)) # (943, 2)

    # project V into 2-D space
    V_tilda = np.transpose(np.dot(np.transpose(A), V))
    print("V shape: " + str(V_tilda.shape)) # (1682, 2)

    return U_tilda, V_tilda

In [12]:
U, V = matrix_factor(model, train_Y)

U shape: (943, 20)
V shape: (20, 1682)


In [13]:
U_tilda, V_tilda = SVD(U, V)

A shape: (20, 2)
U shape: (943, 2)
V shape: (1682, 2)


In [41]:
def get_err(U, V, Y, A, B, y_mean, reg=0.0):
    """
    Takes as input a matrix Y of triples (i, j, Y_ij) where i is the index of a user,
    j is the index of a movie, and Y_ij is user i's rating of movie j and
    user/movie matrices U and V.

    Returns the mean regularized squared-error of predictions made by
    estimating Y_{ij} as the dot product of the ith row of U and the jth column of V^T.
    """
    # Compute mean squared error on each data point in Y; include
    # regularization penalty in error calculations.
    # We first compute the total squared squared error
    err = 0.0
    for (i,j,Yij) in Y:
        err += 0.5 *(Yij - y_mean - np.dot(U[i-1], V[:,j-1]) - A[i-1] - B[:,j-1])**2
    # Add error penalty due to regularization if regularization
    # parameter is nonzero
    if reg != 0:
        U_frobenius_norm = np.linalg.norm(U, ord='fro')
        V_frobenius_norm = np.linalg.norm(V, ord='fro')
        A_frobenius_norm = np.linalg.norm(A, ord='fro')
        B_frobenius_norm = np.linalg.norm(B, ord='fro')
        err += 0.5 * reg * (U_frobenius_norm ** 2)
        err += 0.5 * reg * (V_frobenius_norm ** 2)
        err += 0.5 * reg * (A_frobenius_norm ** 2)
        err += 0.5 * reg * (B_frobenius_norm ** 2)
    # Return the mean of the regularized error
    return err / float(len(Y))

In [16]:
Y_train = np.loadtxt('data/train.txt').astype(int)
y_mean = np.mean(np.array([tup[2] for tup in Y_train]))

In [35]:
model.bu = model.bu.reshape(943,1)
model.bi = model.bi.reshape(1682,1)

In [39]:
np.transpose(model.bi).shape

(1, 1682)

In [42]:
err = get_err(U, V, Y_train, model.bu, np.transpose(model.bi), y_mean, reg=0.1)

In [43]:
err

array([0.84054055])